<a href="https://colab.research.google.com/github/lolobq/ML_Final_Project/blob/main/Data%20Manipulation/Enumerated_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

NIBRS = pd.read_csv('NIBRS_Code.csv')

df = pd.read_csv('merged_datasets_FIXED.csv')

filtered_df = df.loc[
    (df['ADDRESS_DESCRIPTION'] != 'Location where officer took report') &
    (df['CLEARANCE_STATUS'] != 'Unfounded')
].drop(columns=['DATE_INCIDENT_END','ADDRESS_DESCRIPTION', 'YEAR', 'LOCATION', 'DATE_REPORTED', 'X_COORD_PUBLIC', 'Y_COORD_PUBLIC', 'HIGHEST_NIBRS_CODE', 'DIVISION_ID', 'CLEARANCE_DETAIL_STATUS', 'CLEARANCE_STATUS', 'CLEARANCE_DATE'])
filtered_df['VulnerableBinary'] = filtered_df['Vulnerable'].map({'No': 0, 'Yes': 1})
filtered_df = filtered_df.drop(columns=['Vulnerable'])
# Print the filtered DataFrame
filtered_df = pd.merge(filtered_df, NIBRS, on='HIGHEST_NIBRS_DESCRIPTION', how='left')
# Add a new column 'Is_Violent_Crime' based on the value in the 'Crime Against' column
filtered_df['Violent-Crime'] = (filtered_df['Crime Against'] == 'Person').astype(int)
filtered_df.loc[filtered_df['Violent-Crime'] == 0, 'Violent-Crime'] = 0

columns_to_enum = ['PLACE_DETAIL_DESCRIPTION', 'LOCATION_TYPE_DESCRIPTION', 'PLACE_TYPE_DESCRIPTION', 'HIGHEST_NIBRS_DESCRIPTION', 'CMPD_PATROL_DIVISION', 'Crime Against']
enum_mapping = {}

# Enumerate specified columns and store the mapping
for column in columns_to_enum:
    filtered_df[f'{column}'], unique_codes = pd.factorize(filtered_df[column])

    # Print the mapping of codes to original values
    print(f"Mapping of {column} codes:")
    print(dict(zip(unique_codes, range(0, len(unique_codes)))))

# Replace '%' in the entire DataFrame
df_clean = filtered_df.replace('%', '', regex=True)

# Now df_cleaned contains the DataFrame with rows having missing values in 'Crime Against' column removed
df_clean['DATE_INCIDENT_BEGAN'] = pd.to_datetime(df_clean['DATE_INCIDENT_BEGAN'], errors='coerce').dt.date
df_clean = df_clean.dropna()
df_clean['DATE_INCIDENT_BEGAN'] = df_clean['DATE_INCIDENT_BEGAN'].astype(str).str.replace('-', '')
df_clean = df_clean.drop(columns=['Offense Code'])
df_clean['ZIP'] = pd.to_numeric(df_clean['ZIP'], errors='coerce')
df_clean = df_clean.dropna()
print(df_clean.head())
print("Original number of rows:", df_clean.shape[0])
